In [2]:
import numpy as np
import pandas as pd
import xarray as xr

## Setup

In [26]:
study_timeperiod = pd.date_range(start="2014-09", end="2017-04", freq="1M").drop(["2015-05-31"])
print(f"Start: {study_timeperiod[0]}, End: {study_timeperiod[-1]}")
print("NOTE: 2015-05 removed")

Start: 2014-09-30 00:00:00, End: 2017-03-31 00:00:00
NOTE: 2015-05 removed


## Inputs

In [27]:
# mozart
file_paths = []
for month in study_timeperiod:
    yyyy, mm, _ = str(month).split("-")
    file_paths.append(
        f"../1_transport/intermediates/MOZART/output/BasisFnsUpdated/{yyyy}{mm}/"
        f"BasisFnsUpdated.mz4.h0.{yyyy}-{mm}-01-03600.nc"
    )


In [1]:
# # concatenate CO2_SRF_EMIS_avrg variable from MOZART diagnostics
# def extract_co2_emission_mozart(ds):
#     return ds["CO2_SRF_EMIS_avrg"]

# ds = xr.open_mfdataset(file_paths, preprocess=extract_co2_emission_mozart, decode_times=False)
# ds


# Issues: 
# - In each "yyyymm" folder (under MOZART/output/BasisFnsUpdated), there are 4 h0 files -- the month indicated by the folder, and for the following three months. If I want to produce a continuous version of the CO2_SRF_EMIS_avrg variable, should I be pulling that from the h0 file for the month indicated by the corresponding folder? Also, what do the files for the other three months represent?
# - Folders for 2015-05 through 2016-01 (inclusive) are missing
# - The "yyyymm" folders (under MOZART/output/BasisFnsUpdated) start with 201409 and end with 201703, but the GEOS-Chem equivalents (HEMCO_diagnostics files under GEOS_Chem/runs/run.v12.3.2.base/output) run from 201409 through 201706 (with no interruption). Do we need to account for the difference of three months at the end when comparing these diagnostics? Would that come from the other three files in the MOZART/output/BasisFnsUpdated/201703 directory?

In [6]:
# geos chem
def extract_co2_emission_geoschem(ds):
    return ds["EmisCO2_Total"]

geos_chem_path_glob = "../1_transport/intermediates/GEOS_Chem/runs/run.v12.3.2.base/output/HEMCO_diagnostics.*.nc"
with xr.open_mfdataset(geos_chem_path_glob, preprocess=extract_co2_emission_geoschem) as ds:
    da = (
        # compute the sum over the pressure levels
        ds["EmisCO2_Total"].sum(dim=["lev"])
        # temporal interval is hourly, multiply by 3600 (s/hr)
        # multiply by X (m^2 / grid cell)
    )

da

<xarray.DataArray 'EmisCO2_Total' (time: 24816, lat: 91, lon: 144)>
dask.array<sum-aggregate, shape=(24816, 91, 144), dtype=float32, chunksize=(744, 91, 144), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 2014-09-01T00:30:00 ... 2017-06-30T23:30:00
  * lon      (lon) float64 -180.0 -177.5 -175.0 -172.5 ... 172.5 175.0 177.5
  * lat      (lat) float64 -89.5 -88.0 -86.0 -84.0 -82.0 ... 84.0 86.0 88.0 89.5

In [7]:
ds.time

<xarray.DataArray 'time' (time: 24816)>
array(['2014-09-01T00:30:00.000000000', '2014-09-01T01:30:00.000000000',
       '2014-09-01T02:30:00.000000000', ..., '2017-06-30T21:30:00.000000000',
       '2017-06-30T22:30:00.000000000', '2017-06-30T23:30:00.000000000'],
      dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 2014-09-01T00:30:00 ... 2017-06-30T23:30:00
Attributes:
    standard_name:  time
    long_name:      Time
    axis:           T

In [9]:
da.lon

<xarray.DataArray 'lon' (lon: 144)>
array([-180. , -177.5, -175. , -172.5, -170. , -167.5, -165. , -162.5, -160. ,
       -157.5, -155. , -152.5, -150. , -147.5, -145. , -142.5, -140. , -137.5,
       -135. , -132.5, -130. , -127.5, -125. , -122.5, -120. , -117.5, -115. ,
       -112.5, -110. , -107.5, -105. , -102.5, -100. ,  -97.5,  -95. ,  -92.5,
        -90. ,  -87.5,  -85. ,  -82.5,  -80. ,  -77.5,  -75. ,  -72.5,  -70. ,
        -67.5,  -65. ,  -62.5,  -60. ,  -57.5,  -55. ,  -52.5,  -50. ,  -47.5,
        -45. ,  -42.5,  -40. ,  -37.5,  -35. ,  -32.5,  -30. ,  -27.5,  -25. ,
        -22.5,  -20. ,  -17.5,  -15. ,  -12.5,  -10. ,   -7.5,   -5. ,   -2.5,
          0. ,    2.5,    5. ,    7.5,   10. ,   12.5,   15. ,   17.5,   20. ,
         22.5,   25. ,   27.5,   30. ,   32.5,   35. ,   37.5,   40. ,   42.5,
         45. ,   47.5,   50. ,   52.5,   55. ,   57.5,   60. ,   62.5,   65. ,
         67.5,   70. ,   72.5,   75. ,   77.5,   80. ,   82.5,   85. ,   87.5,
         90. ,   92.5,   95. ,   97.5,  100. ,  102.5,  105. ,  107.5,  110. ,
        112.5,  115. ,  117.5,  120. ,  122.5,  125. ,  127.5,  130. ,  132.5,
        135. ,  137.5,  140. ,  142.5,  145. ,  147.5,  150. ,  152.5,  155. ,
        157.5,  160. ,  162.5,  165. ,  167.5,  170. ,  172.5,  175. ,  177.5])
Coordinates:
  * lon      (lon) float64 -180.0 -177.5 -175.0 -172.5 ... 172.5 175.0 177.5
Attributes:
    standard_name:  longitude
    long_name:      Longitude
    units:          degrees_east
    axis:           X